In [5]:
#model
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from PyQt5.QtCore import pyqtSignal, QThread

class global_model:
    #def change_time_to(self, pos_in_time):
    #    raise NotImplementedError
    def add_observer(self, observer):
        raise NotImplementedError
    
class video_display(global_model):
    total_frames = None
    observers = set()
    cap = None
    filepath = ""
    
    def __init__(self, filepath = None):
        self.current_frame = 10
        self.filepath = filepath
        self.cap = cv2.VideoCapture(self.filepath)
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.factor = self.total_frames//10000
        
    def add_observer(self, observer):
        self.observers.add(observer)
    
    def notify_observers(self):
        for observer in self.observers:
            observer.update()
            
    def change_time_to(self, pos_in_time):
        self.set_current_video_frame(pos_in_time//self.factor)
        self.notify_observers()
        

    def get_frame(self):
        self.cap.set(1,self.current_frame)
        ret, frame = self.cap.read()
        return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #return self.cap.retrieve(self.current_frame)[1]
        
    def set_current_video_frame(self, x):
        #print("The position in the video is " + str(x))
        if(x > self.total_frames):
            self.current_frame = self.total_frames-1
        else:
            self.current_frame = x
        

    def get_amount_of_frames():
        return self.total_frames
    
    def add_observer(self, observer):
        self.observers.add(observer)
    
    def notify_observers(self):
        for observer in self.observers:
            observer.update()
    
class eeg_display(global_model):
    data = None
    filepath = None
    title = None
    channel = None
    updated = pyqtSignal()
    observers = set()
    is_deleted = False #When model is to be removed
    
    def get_channel(self):
        return self.channel
    
    def set_channel(self, channel):
        self.channel = channel
        self.load_eeg_file()
        self.notify_observers()
        
    def set_filepath(self, filepath):
        self.filepath = filepath
        self.set_title(os.path.basename(self.filepath) + "    Channel " + str(self.channel))
    
    def set_title(self, title):
        self.title = title
    
    def get_title(self):
        return self.title
    
    def get_filepath(self, filepath):
        return self.filepath
    
    def get_data(self):
        return self.data
    
    def delete(self):
        self.is_deleted = True
        self.notify_observers()
    
    def deleted(self):
        return self.is_deleted
    
    def __init__(self, filepath = None, channel = 0):
        self.filepath = filepath
        self.is_deleted = False
        
        if(channel!=None):
            self.channel = channel
        
        if(filepath != None):
                self.set_filepath(filepath)
                self.load_eeg_file()
        else:
            self.filepath = "Load data...!"
            self.data = np.zeros(100)
        self.set_title(os.path.basename(self.filepath) + "    Channel " + str(self.channel) )
    
    def add_observer(self, observer):
        self.observers.add(observer)
    
    def notify_observers(self):
        for observer in self.observers:
            observer.update()
            
    def change_channel(self, channel):
        self.channel = channel
        self.load_eeg_file()
        self.notify_observers(self)
                    
    def load_eeg_file(self):
        if(self.filepath==None):
            print("No filepath set")
            return None
        
        n_channels = 64
        bytes_per_sample = 2 #Because int16

        my_type = np.dtype([("channel"+str(x),np.int16) for x in range(0,n_channels)])
        byte_size = os.path.getsize(self.filepath)

        nFrames =  byte_size // (bytes_per_sample * n_channels);
        self.data = np.fromfile(self.filepath,dtype=my_type)["channel"+str(self.channel)]
        
        self.notify_observers()

In [6]:
from PyQt5.QtWidgets import QFileDialog
from PyQt5.QtWidgets import QWidget

class eeg_display_controller(QWidget):
        def __call__(self):
            self.open()

        def __init__(self, model):
            super().__init__()
            self.model = model
            
        def open(self):
            options = QFileDialog.Options()
            options |= QFileDialog.DontUseNativeDialog
            fileName, _ = QFileDialog.getOpenFileName(self,"QFileDialog.getOpenFileName()", "","All Files (*);;Python Files (*.py)", options=options)
            if fileName:
                self.model.set_filepath(fileName)
                self.model.load_eeg_file()

In [7]:
from PyQt5 import QtGui,QtCore
import sys
import numpy as np
import pylab
import time
import pyqtgraph
from PyQt5.QtGui import QGraphicsView
from pyqtgraph import PlotWidget, PlotItem
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import (QSizePolicy, QSlider)
from PyQt5.QtWidgets import (QApplication, QCheckBox, QGridLayout, QGroupBox,
        QMenu, QPushButton, QRadioButton, QVBoxLayout, QWidget, QLabel, QSpinBox)


import random

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt


def _translate(context, text, disambig):
    return QtGui.QApplication.translate(context, text, disambig)

class MainApp(QtGui.QMainWindow):
    eeg_displays = [] #type dataDisplayWithOptions
    model = []
    
    def __init__(self, data_models, video, parent=None):
        super(MainApp, self).__init__(parent)
        self.model = data_models#type eeg display
        self.video = video#type video display
        
        # Subscribe: Add self to all models that will be displayed in it (i.e. all models)
        for m in self.model:
            m.add_observer(self)
        self.video.add_observer(self)
        
        #Draw User Interface
        pyqtgraph.setConfigOption('background', 'w') #White background
        self.resize(1280, 640)
        app.aboutToQuit.connect(app.deleteLater)

        self.setupUi()
        
        """        for m in self.model:
            if(not m== None):
                self.sld.valueChanged.connect(m.change_time_to)"""
            
    def add_data_display(self):
        #Create new MODEL and append it to self.model
        new_display = eeg_display()
        self.model.append(new_display)
        
        #Subscribe
        new_display.add_observer(self)
        
        #Create VIEW and add it (widget) to layout
        optiondisplay = dataDisplayWithOptions(new_display)
        self.eeg_displays.append(optiondisplay)
        self.verticalLayout.addWidget(optiondisplay)
        
        #redraw Add button at bottom
        self.verticalLayout.removeWidget(self.addDataDisplays)
        self.verticalLayout.addWidget(self.addDataDisplays)

    def setupUi(self):
        self.centralwidget = QtGui.QWidget(self)
        self.verticalLayout = QtGui.QVBoxLayout(self.centralwidget)
         
        self.verticalLayout.removeWidget
        self.video_view = VideoPlot(self.video)
        self.verticalLayout.addWidget(self.video_view)#ADD VIDEO HERE
        self.video_view.update()
        
        self.sld = QSlider(Qt.Horizontal, self)
        self.sld.setRange(0, 10000)
        self.verticalLayout.addWidget(self.sld)
        self.sld.setTickPosition(QSlider.TicksAbove)

        self.sld.valueChanged.connect(self.video.change_time_to)
        
        #Draw (+) button to add data displays
        # Connect navigation elements to respective slots

        self.addDataDisplays = QtGui.QPushButton(self.centralwidget)
        self.verticalLayout.addWidget(self.addDataDisplays)
        self.addDataDisplays.setText("+")
        self.addDataDisplays.clicked.connect(self.add_data_display)
        
        #Draw EEG models
        self.draw_eeg_models()

        self.setCentralWidget(self.centralwidget)

        self.setWindowTitle(QtGui.QApplication.translate("EEG Viewer", "EEG Viewer", None))

        QtCore.QMetaObject.connectSlotsByName(self)
    
    def update(self):
        self.video_view.update()
        
        #Update all views
        for m,view,count in zip(self.model,self.eeg_displays,range(0,len(self.eeg_displays))):
            if(type(m) == type(eeg_display())):
                if(m.deleted()):
                    del(self.eeg_displays[count])
                    del(self.model[count])
                    self.setupUi()
                    view.update()
                else:
                    view.update()
                    


    def draw_eeg_models(self):
        self.eeg_displays=[]#Make sure list is empty BEEEEEFORE redrawing!!!
        
        for m in self.model:
            if(type(m) == type(eeg_display())):
                wrapped = dataDisplayWithOptions(m)
                
                self.eeg_displays.append(wrapped)
                self.verticalLayout.addWidget(wrapped)
                self.verticalLayout.removeWidget(self.addDataDisplays)
                self.verticalLayout.addWidget(self.addDataDisplays)


class dataDisplayWithOptions(QWidget):

    def __init__(self, model, parent=None):
        super(QtGui.QWidget,self).__init__(parent)
        self.model = model
        self.horizontalLayout = QtGui.QHBoxLayout()
        self.horizontalLayout.setSpacing(20)
        self.setLayout(self.horizontalLayout)####IMPORTANT
        
        self.setMinimumSize(120, 120)
        
        self.grPlot = InteractiveDataPlot(self.model, self)
        self.horizontalLayout.addWidget(self.grPlot)
        
        self.horizontalLayout.addWidget(self.createOptionsGroup())
    
    def update(self):
        self.grPlot.plot()
        self.l1.setText("Channel: " + str(self.model.get_channel()))
        self.groupBox.setTitle(self.model.get_title())

        
    def createOptionsGroup(self):
        self.groupBox = QGroupBox(self.model.get_title())
        self.groupBox.setAlignment(4)
        
        self.load_button = QtGui.QPushButton()
        self.close_button = QtGui.QPushButton()

        self.l1 = QLabel("Channel: " + str(self.model.get_channel()))
        self.spin_box = QSpinBox()
        
        vbox = QVBoxLayout()
        vbox.addWidget(self.l1)
        vbox.addWidget(self.spin_box)
        vbox.addWidget(self.load_button)
        vbox.addWidget(self.close_button)
        
        self.load_button.setText("Load...")
        self.close_button.setText("Close")
        
        #USE EEG DISPLAY CONTROLLER TO HAVE THE MODEL LOAD ITS DATA
        loader = eeg_display_controller(self.model)
        self.load_button.clicked.connect(loader)
        
        #LET THE MODEL COMMUNICATE IT'S DEAD
        self.close_button.clicked.connect(self.delete)
        
        #Use spin box to switch through channels
        self.spin_box.valueChanged.connect(self.model.set_channel)
        
        vbox.addStretch(1)
        self.groupBox.setLayout(vbox)

        return self.groupBox
    
    def delete(self):
        self.model.delete()

class VideoPlot(QGraphicsView):
    def __init__(self, video, parent=None):
        super(VideoPlot, self).__init__(parent)
        self.video = video
        self.parent=parent
        self.figure = plt.figure()
        self.canvas = FigureCanvas(self.figure)
        self.setMinimumSize(120, 210)
        
        #Create Layout and add self
        layout = QVBoxLayout()
        layout.addWidget(self.canvas)
        self.setLayout(layout)
        self.canvas.show()

    def update(self):
        self.figure.clear()
        self.axes=self.figure.add_subplot(1,1,1)
        frame = self.video.get_frame()
        im = self.axes.imshow(frame)
        self.figure.subplots_adjust(left=0.0,bottom=0.0, top=1.0, right = 1.0)#Dont waste space
        self.axes.axis('off')
        self.canvas.draw()
        self.canvas.show()

        
class DataPlot(FigureCanvas):
    def __init__(self, model, parent=None, width=5, height=4, dpi=100):
        self.model = model
        
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
 
        FigureCanvas.__init__(self, fig)
        self.setParent(parent)
 
        FigureCanvas.setSizePolicy(self,QSizePolicy.Expanding,QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)
        self.plot()
 
    def plot(self):
        data = self.model.get_data()
        self.figure.clear()
        ax = self.figure.add_subplot(111)
        ax.plot(data, 'r-')
        ax.set_title(self.model.get_title())
        self.draw()
        
class InteractiveDataPlot(PlotWidget):
    def __init__(self, model, parent=None, width=5, height=4, dpi=100):
        super().__init__(parent)
        self.model = model
        self.plot()
 
    def plot(self):
        data = self.model.get_data()
        X=np.arange(len(data))
        C=pyqtgraph.hsvColor(1)
        pen=pyqtgraph.mkPen(color=C,width=1)
        self.getPlotItem().plot(X,data,pen=pen,clear=True)
        #PlotItem.plot(X,data,pen=pen,clear=True)
        

In [ ]:
if __name__=="__main__":
    eeg_models =  [eeg_display("/data/pt_01843/eegData/DualEEG_RPS_rawData/DualEEG_RPS_C_02.eeg",0),
                  eeg_display("/data/pt_01843/eegData/DualEEG_RPS_rawData/DualEEG_RPS_C_02.eeg",1) ] #For now SUBJECT TO CHANGE
    video = video_display("/data/p_01888/Video/coSMIC_all/Pilot_1/coSMIC_all_P01_C1.wmv")
    app = QtGui.QApplication(sys.argv)
    form = MainApp(eeg_models, video)
    form.show()
    form.update() #start with something
    app.exec_()
    print("DONE")